## 实验：输出层激活函数 Softmax

### 实验概要

神经网络可以用在分类问题和回归问题上，不过需要根据情况改变输出层的激活函数。一般而言 ——

- 回归问题用 **恒等函数**
- 分类问题用 **softmax 函数**

两者在输出层的输入到输出的转换对比如下：

<img src="./img/1_05.png" width="40%">

#### softmax 函数

在数学，尤其是概率论和相关领域中，softmax 函数，是逻辑函数 **sigmoid** 的一种推广。它能将一个含任意实数的 K 维向量 **压缩** 到另一个 K 维实向量中，使得每一个元素的范围都在 0-1 之间，并且使所有元素的和为 1。这样，每个元素就代表了属于某个分类的概率。值越大，概率就越大，属于某一类的可能性就越大。

<img src="./img/3_05.png" width="50%">

因此，softmax 广泛应用于机器学习和深度学习中的多分类问题。在深度学习中，softmax 常用于多分类问题最后一层的激活函数，用于输出某样本属于某个分类的概率值。softmax 函数可以用以下公式表示：

$$
y_{k}=\frac{\exp \left(a_{k}\right)}{\sum_{i=1}^{n} \exp \left(a_{i}\right)}
$$

***exp(x)*** 是表示 ***e<sup>x</sup>*** 的指数函数（***e*** 是纳皮尔常数 2.7182 ...）。上式表示，假设输出层共有 ***n*** 个神经元，计算第 ***k*** 个神经元的输出 ***y<sub>k</sub>***。softmax 函数的分子是输入信号 ***a<sub>k</sub>*** 的指数函数，分母是所有输入信号的指数函数的和。

### 实验目标

本实验中，我们将通过 Python 实现 softmax 函数，并且通过数学推理与代码对 softmax 函数的实现方式进行改进。

### 1. softmax 函数基本实现

根据 softmax 函数公式：

$$
y_{k}=\frac{\exp \left(a_{k}\right)}{\sum_{i=1}^{n} \exp \left(a_{i}\right)}
$$

借助 numpy 内置 **`np.exp()`** 指数函数，可以很直观的写出 softmax 函数的 python 代码： 

In [1]:
import numpy as np

a = np.array([0.3, 2.9, 4.0])

# 指数函数运算，对纳皮尔常数 2.7182... 分别使用 0.3，2.9，4.0 求幂
exp_a = np.exp(a) 
# 输出指数函数结果，即 softmax 函数分子
print(exp_a)      
# 输出 "[ 1.34985881 18.17414537 54.59815003]"

[ 1.34985881 18.17414537 54.59815003]


In [2]:
# 指数函数的和，即 1.34985881+18.17414537+54.59815003
sum_exp_a = np.sum(exp_a) 
# 输出指数函数的和，即 softmax 函数分母
print(sum_exp_a)          

74.1221542101633


In [3]:
y = exp_a / sum_exp_a    
# 得出 softmax 输出
print(y)                  

[0.01821127 0.24519181 0.73659691]


### 2. 定义 softmax 为 Python函数

将上面的代码转换为自定义的 python 函数 softmax()，便于以后直接调用。

In [4]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
        
    return y

### 3. 避免 softmax 溢出

上面的 softmax 函数的实现虽然正确描述了 softmax 的公式，但在计算机的运算上有一定的缺陷。这个缺陷就是溢出问题。 softmax 函数的实现中要进行指
数函数的运算，但是此时指数函数的值很容易变得非常大。比如， ***e<sup>10</sup>*** 的值会超过 20000，***e<sup>100</sup>*** 会变成一个后面有 40 多个 0 的超大值， ***e<sup>1000</sup>*** 的结果会返回一个表示无穷大的 inf。如果在这些超大值之间进行除法运算，结果会出现“不确定”的情况。由于计算机可以表示的数值范围是有限的。因此，会出现超大值无法表示的问题。这个问题称为 **溢出**。

softmax 函数的实现可以改进为：

$$
\begin{aligned}
y_{k}=\frac{\exp \left(a_{k}\right)}{\sum_{i=1}^{n} \exp \left(a_{i}\right)} &=\frac{\mathrm{C} \exp \left(a_{k}\right)}{\mathrm{C} \sum_{i=1}^{n} \exp \left(a_{i}\right)} \\
&=\frac{\exp \left(a_{k}+\log \mathrm{C}\right)}{\sum_{i=1}^{n} \exp \left(a_{i}+\log \mathrm{C}\right)} \\
&=\frac{\exp \left(a_{k}+\mathrm{C}^{\prime}\right)}{\sum_{i=1}^{n} \exp \left(a_{i}+\mathrm{C}^{\prime}\right)}
\end{aligned}
$$

首先，在分子和分母上都乘上 ***C*** 这个任意的常数（因为同时对分母和分子乘以相同的常数，所以计算结果不变）。然后，把这个 ***C*** 移动到指数函数（***exp***）中，记为 ***log C***。最后，把 ***log C*** 替换为另一个符号 ***C'*** 。在进行 softmax 的指数函数的运算时，加上（或者减去）某个常数并不会改变运算的结果。这里的 ***C'*** 可以使用任何值，但是为了防止溢出，一般会使用输入信号中的最大值。

我们对比一下效果：首先，执行原有的 softmax 运算 ——

In [5]:
a = np.array([1010, 1000, 990])
# softmax函数的运算
np.exp(a) / np.sum(np.exp(a)) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([nan, nan, nan])

从上面的结果可知，由于指数太大，运算没有被正确执行：

![](./img/2_05.png)

`array([nan, nan, nan])` # 输出了无效的返回值

运行上面定义的 softmax 函数，会得到相同的溢出警告，与返回无效值。

In [6]:
# 运行softmax函数
softmax(a)  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


array([nan, nan, nan])

In [7]:
# 设置一个输入数组中的最大值，这里是：1010
c = np.max(a) 
# 重新运算输入数组
a - c         

array([  0, -10, -20])

再次运行 softmax 运算，可以得到正确的输出：

In [8]:
np.exp(a - c) / np.sum(np.exp(a - c))

array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

根据上面的逻辑，重新定义 softmax 的 python 函数并运行，可以得到正确的返回值：

In [9]:
# 重新定义softmax函数
def softmax(a):          
    c = np.max(a)
    # 溢出对策
    exp_a = np.exp(a - c) 
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

# 重新运行softmax函数
softmax(a) 

array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

### 4. softmax 函数的特征

如下面代码所示， softmax 函数的输出是 0.0 到 1.0 之间的实数。并且，softmax 函数的输出值的总和是1。输出总和为 1 是 softmax 函数的一个重要性质。正因为有了这个性质，我们才可以把 softmax 函数的输出解释为 **概率**。

In [10]:
a = np.array([0.3, 2.9, 4.0])
y = softmax(a)

print(y)
np.sum(y)

[0.01821127 0.24519181 0.73659691]


1.0

上面的例子可以解释成：

- `y[0]` 的概率是 `0.018（1.8%）`
- `y[1]` 的概率是 `0.245（24.5%）`
- `y[2]` 的概率是 `0.737（73.7%）`

从概率的结果来看，可以说：

> 因为第三个元素（y[2]）的概率最高，所以答案是第三个类别（y[2]）。

这里需要注意的是，即便使用了softmax 函数，各个元素之间的大小关系也不会改变。这是因为指数函数（***y = exp(x)***）是单调递增函数。实际上，上例中 a 的各元素的大小关系和 y 的各元素的大小关系并没有改变。比如，a 的最大值是第 2 个元素， y 的最大值也仍是第 2 个元素。一般而言，神经网络只把输出值最大的神经元所对应的类别作为识别结果。并且，即便使用 softmax 函数，输出值最大的神经元的位置也不会变。因此，神经网络在进行分类推理时（即完成训练后对新的输入数据进行分类判断），输出层的 softmax 函数可以省略。

### 5. sigmoid 与 softmax 对比

假设我们的输出层，从上一层神经网络接收到的输入值为：`[-0.5, 1.2, -0.1, 2.4]`。下一步，是使用某些激活函数（**Sigmoid** 或 **Softmax** 激活函数）将这些原始输出值转换为概率。**Sigmoid** 和 **Softmax** 会产生不同的结果。

#### sigmoid

$$
f(x)=\operatorname{sigmoid}(x)=\frac{1}{1+e^{-x}}
$$

In [11]:
# 定义sigmoid函数
def sigmoid(x):
    return 1.0/(1+np.exp(-x))  

x = np.array([-0.5, 1.2, -0.1, 2.4])
y = sigmoid(x)

print(y)

[0.37754067 0.76852478 0.47502081 0.9168273 ]


In [12]:
np.sum(y)

2.5379135683243006

可以看到：

- Sigmoid 输入值: -0.5, 1.2, -0.1, 2.4
- Sigmoid 输出值: 0.37754067 0.76852478 0.47502081 0.9168273
- Sigmoid 输出总和：2.5379135683243006

sigmoid 产生的概率是独立的，它们不限制于总和的值应该是多少，其原因是 Sigmoid 会分别运算每个原始输出值。

#### softmax

$$
\operatorname{softmax}\left(z_{j}\right)=\frac{e^{z_{j}}}{\sum_{k=1}^{K} e^{z_{k}}} \text { for } j=1, \ldots, K
$$

In [13]:
# 重新定义softmax函数
def softmax(x):          
    c = np.max(x)
    # 溢出对策
    exp_x = np.exp(x - c) 
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    return y

x = np.array([-0.5, 1.2, -0.1, 2.4])
y = softmax(x)

print(y)

[0.03825567 0.20940951 0.05707075 0.69526407]


In [14]:
np.sum(y)

1.0

可以看到：

- softmax 输入值: -0.5, 1.2, -0.1, 2.4
- softmax 输出值: 0.03825567 0.20940951 0.05707075 0.69526407
- softmax 输出总和：1

softmax 输出是相互关联的。通过算法设计，Softmax 概率总和永远为 1。因此，我们想增加一个类别的可能性，则另一个类别必须减少相等的数量。

### 实验小结

在本实验中，你通过 Python 实现 softmax 函数，并且通过数学推理与代码对 softmax 函数的实现方式进行改进，从而避免由于输入值过大或者过小，而造成计算机运算溢出的问题。最后，你对比了使用 sigmoid 和 softmax 作为激活函数之间的区别。

> **sigmoid 函数往往用于处理二分类问题，而 softmax 则往往用于多分类问题。**

另外，对于分类问题，输出层的神经元数量一般设定为类别的数量。比如，对于某个输入图像，预测是图中的数字 0 到 9 中的哪一个的问题（10 类别分类问题），此时，可以将输出层的神经元设定为 10 个。如下图：神经元 y2 颜色最深，输出的值最大 —— 从 softmax 看来，就是概率最高。这表明这个神经网络预测的是 y2 对应的类别，也是 **2**。

<img src="./img/4_05.png" width="50%">